In [5]:
import pandas as pd
from meteostat import Point, Daily
from datetime import datetime, timedelta
import requests
import ssl
from geopy.distance import geodesic

ssl._create_default_https_context = ssl._create_unverified_context



In [6]:



completed_orders= pd.read_csv('../data/processed/nb_cleaned.csv')
delivery_requests = pd.read_csv('../data/processed/location_cleaned.csv')



In [7]:
def parse_coordinates(address):
    """
    Parse the address to extract latitude and longitude.
    :param address: String in the format "latitude, longitude"
    :return: Tuple containing (latitude, longitude)
    """
    # Clean up the address string
    address = address.strip()  # Remove leading and trailing whitespace
    lat, lng = map(float, address.split(','))  # Split and convert to float
    return lat, lng


# Apply the function to the Trip Origin and Trip Destination columns
completed_orders[['Origin Lat', 'Origin Lng']] = completed_orders['Trip Origin'].apply(parse_coordinates).apply(pd.Series)
completed_orders[['Destination Lat', 'Destination Lng']] = completed_orders['Trip Destination'].apply(parse_coordinates).apply(pd.Series)


In [8]:

def calculate_distance(row):
    origin = (row['Origin Lat'], row['Origin Lng'])
    destination = (row['Destination Lat'], row['Destination Lng'])
    return geodesic(origin, destination).kilometers

completed_orders['trip_distance'] = completed_orders.apply(calculate_distance, axis=1)

print(completed_orders.head())


KeyboardInterrupt: 

In [ ]:
completed_orders.to_csv('../data/processed/nb_cleaned.csv', index=False)


Calculating the distance between th driver location and the trip origin for completed trips

In [ ]:
completed_orders.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534368 entries, 0 to 534367
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        534368 non-null  int64  
 1   Trip ID           534368 non-null  int64  
 2   Trip Origin       534368 non-null  object 
 3   Trip Destination  534368 non-null  object 
 4   Trip Start Time   534368 non-null  object 
 5   Trip End Time     534368 non-null  object 
 6   Origin Lat        534368 non-null  float64
 7   Origin Lng        534368 non-null  float64
 8   Destination Lat   534368 non-null  float64
 9   Destination Lng   534368 non-null  float64
 10  trip_distance     534368 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 44.8+ MB


In [ ]:
completed_orders['Trip Origin']
completed_orders[['Origin Lat', 'Origin Lng']] = completed_orders['Trip Origin'].apply(
    lambda x: pd.Series(map(float, x.split(',')))
)


In [ ]:
completed_orders.head()

,Unnamed: 0,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,Origin Lat,Origin Lng,Destination Lat,Destination Lng,trip_distance
0,0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,6.508813,3.377403,6.650970,3.345031,16.123451
1,1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,6.431671,3.455538,6.428081,3.472189,1.884305
2,2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,6.631679,3.338898,6.508324,3.359040,13.822393
3,3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,6.572757,3.367708,6.584881,3.361407,1.511034
4,4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,6.601042,3.276634,6.450107,3.391615,20.984319


In [ ]:
merged_df = delivery_requests.merge(
    completed_orders[['Trip ID', 'Origin Lat', 'Origin Lng']], 
    left_on='order_id', 
    right_on='Trip ID', 
    how='left'
)


In [ ]:
columns_to_drop = ['Unnamed: 0', 'Trip Origin_x', 'origin_lat', 'origin_lng', 'Trip ID_y', 'Trip Origin_y', 'Trip ID_x']
cleaned_df = merged_df.drop(columns=columns_to_drop)

In [ ]:
cleaned_df.head(2)

,id,order_id,driver_id,driver_action,lat,lng,driver_to_order_distance,Trip ID,Origin Lat,Origin Lng
0,1,392001,243828,accepted,6.602207,3.270465,815449.701328,392001.0,6.601042,3.276634
1,2,392001,243588,rejected,6.592097,3.287445,815292.991562,392001.0,6.601042,3.276634


In [10]:

delivery_requests = pd.read_csv('test.csv')

delivery_requests.head(2)
print(delivery_requests.isna().sum())
cleaned_df = delivery_requests.dropna(subset=['lat', 'lng', 'Origin Lat', 'Origin Lng'])



Unnamed: 0                     0
id                             0
order_id                       0
driver_id                      0
driver_action                  0
lat                            0
lng                            0
driver_to_order_distance       0
Trip ID                     2022
Origin Lat                  2022
Origin Lng                  2022
dtype: int64


In [11]:
def calculate_driver_to_order_distance(row):
    driver_location = (row['lat'], row['lng'])
    order_location = (row['Origin Lat'], row['Origin Lng'])
    return geodesic(driver_location, order_location).meters


In [12]:
delivery_requests['driver_to_order_distance'] = cleaned_df.apply(calculate_driver_to_order_distance, axis=1)


In [13]:
delivery_requests.head()

,Unnamed: 0,id,order_id,driver_id,driver_action,lat,lng,driver_to_order_distance,Trip ID,Origin Lat,Origin Lng
0,0,1,392001,243828,accepted,6.602207,3.270465,694.263614,392001.0,6.601042,3.276634
1,1,2,392001,243588,rejected,6.592097,3.287445,1551.694105,392001.0,6.601042,3.276634
2,2,3,392001,243830,rejected,6.596133,3.281784,786.776508,392001.0,6.601042,3.276634
3,3,4,392001,243539,rejected,6.596142,3.280526,692.053650,392001.0,6.601042,3.276634
4,4,5,392001,171653,rejected,6.609232,3.288800,1621.848107,392001.0,6.601042,3.276634


In [15]:
driver_location = (6.602207,3.270465 )
order_location = (	6.601042, 3.276634)

# Calculate the distance in meters
distance = geodesic(driver_location, order_location).meters
print(distance)


694.2656653179304


In [16]:
delivery_requests.to_csv('delivery_requests_with_distances.csv', index=False)


In [17]:
delivery_requests = pd.read_csv('delivery_requests_with_distances.csv')
delivery_requests.head(3)

,Unnamed: 0,id,order_id,driver_id,driver_action,lat,lng,driver_to_order_distance,Trip ID,Origin Lat,Origin Lng
0,0,1,392001,243828,accepted,6.602207,3.270465,694.263614,392001.0,6.601042,3.276634
1,1,2,392001,243588,rejected,6.592097,3.287445,1551.694105,392001.0,6.601042,3.276634
2,2,3,392001,243830,rejected,6.596133,3.281784,786.776508,392001.0,6.601042,3.276634


In [21]:
delivery_requests.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 11 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Unnamed: 0                1557740 non-null  int64  
 1   id                        1557740 non-null  int64  
 2   order_id                  1557740 non-null  int64  
 3   driver_id                 1557740 non-null  int64  
 4   driver_action             1557740 non-null  object 
 5   lat                       1557740 non-null  float64
 6   lng                       1557740 non-null  float64
 7   driver_to_order_distance  1555718 non-null  float64
 8   Trip ID                   1555718 non-null  float64
 9   Origin Lat                1555718 non-null  float64
 10  Origin Lng                1555718 non-null  float64
dtypes: float64(6), int64(4), object(1)
memory usage: 130.7+ MB


In [22]:
completed_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534368 entries, 0 to 534367
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        534368 non-null  int64  
 1   Trip ID           534368 non-null  int64  
 2   Trip Origin       534368 non-null  object 
 3   Trip Destination  534368 non-null  object 
 4   Trip Start Time   534368 non-null  object 
 5   Trip End Time     534368 non-null  object 
 6   Origin Lat        534368 non-null  float64
 7   Origin Lng        534368 non-null  float64
 8   Destination Lat   534368 non-null  float64
 9   Destination Lng   534368 non-null  float64
 10  trip_distance     534368 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 44.8+ MB


In [23]:
merged_df = pd.merge(delivery_requests, completed_orders[['Trip ID', 'Trip Start Time', 'Trip End Time', 'trip_distance']],
                    on='Trip ID', how='left')

In [25]:
merged_df.loc[merged_df['driver_action'] == 'rejected', ['Trip Start Time', 'Trip End Time', 'trip_distance']] = 0


In [29]:
merged_df['Trip Start Time'] = pd.to_datetime(merged_df['Trip Start Time'])
merged_df['Trip End Time'] = pd.to_datetime(merged_df['Trip End Time'])


In [32]:
merged_df['time_taken'] = (merged_df['Trip End Time'] - merged_df['Trip Start Time']).dt.total_seconds()
merged_df['speed'] = merged_df['trip_distance'] / merged_df['time_taken']


,Unnamed: 0,id,order_id,driver_id,driver_action,lat,lng,driver_to_order_distance,Trip ID,Origin Lat,Origin Lng,Trip Start Time,Trip End Time,trip_distance,time_taken,speed
0,0,1,392001,243828,accepted,6.602207,3.270465,694.263614,392001.0,6.601042,3.276634,2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,217.0,0.096702
1,1,2,392001,243588,rejected,6.592097,3.287445,1551.694105,392001.0,6.601042,3.276634,1970-01-01 00:00:00,1970-01-01 00:00:00,0.000000,0.0,NaN
